In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import warnings
warnings.filterwarnings('ignore')

import json
from pandas.io.json import json_normalize

In [2]:
equip = pd.read_csv('../data/equipment.csv')

In [4]:
equip.head()

,Unnamed: 0,_id,index,name,equipment_category,gear_category,cost,weight,url,desc,tool_category,vehicle_category,quantity,weapon_category,weapon_range,category_range,damage,range,properties,2h_damage,armor_category,armor_class,str_minimum,stealth_disadvantage,contents,speed,capacity,throw_range,special
0,0,5f2329a90b1bb138c5940de2,abacus,Abacus,"{'name': 'Adventuring Gear', 'url': '/api/equi...",Standard Gear,"{'quantity': 2, 'unit': 'gp'}",2.0,/api/equipment/abacus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5f2329a90b1bb138c5940de3,acid-vial,Acid (vial),"{'name': 'Adventuring Gear', 'url': '/api/equi...",Standard Gear,"{'quantity': 25, 'unit': 'gp'}",1.0,/api/equipment/acid-vial,"['As an action, you can splash the contents of...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5f2329a90b1bb138c5940de4,alchemists-fire-flask,Alchemist's fire (flask),"{'name': 'Adventuring Gear', 'url': '/api/equi...",Standard Gear,"{'quantity': 50, 'unit': 'gp'}",1.0,/api/equipment/alchemists-fire-flask,"['This sticky, adhesive fluid ignites when exp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5f2329a90b1bb138c5940e51,alchemists-supplies,Alchemist's supplies,"{'name': 'Tools', 'url': '/api/equipment-categ...",NaN,"{'quantity': 50, 'unit': 'gp'}",8.0,/api/equipment/alchemists-supplies,"[""These special tools include the items needed...",Artisan's Tools,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5f2329a90b1bb138c5940dea,amulet,Amulet,"{'name': 'Adventuring Gear', 'url': '/api/equi...",Holy Symbol,"{'quantity': 5, 'unit': 'gp'}",1.0,/api/equipment/amulet,['A holy symbol is a representation of a god o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Equipment cleaning

## Equipment_category 

In [5]:
equip.equipment_category[0]

"{'name': 'Adventuring Gear', 'url': '/api/equipment-categories/adventuring-gear'}"

In [6]:
a = [json.loads(i.replace("'", '"')) for i in equip.equipment_category]

In [7]:
abc=[]

for i in a:
    abc.append(i['name'])

equip['category']= abc

## Cost

In [8]:
equip.cost[0]

"{'quantity': 2, 'unit': 'gp'}"

In [9]:
a = [json.loads(i.replace("'", '"')) for i in equip.cost]

In [10]:
abc=[]

for i in a:
    abc.append(str(i['quantity'])+' '+i['unit'])

equip = equip.drop('cost', axis=1)
equip['cost']=abc

## Description

In [11]:
equip.desc[1]

"['As an action, you can splash the contents of this vial onto a creature within 5 feet of you or throw the vial up to 20 feet, shattering it on impact. In either case, make a ranged attack against a creature or object, treating the acid as an improvised weapon.', 'On a hit, the target takes 2d6 acid damage.']"

In [12]:
equip.desc.fillna('None', inplace=True)

In [13]:
desc=[]

for i in equip.desc:
    if len(i) == 4:
        desc.append('None')
    else:
        i=i.replace("[","").replace("]","").replace("'","").replace('"',"")
        desc.append(i)

equip = equip.drop('desc', axis=1)
equip['desc']=desc

## Damage 

In [14]:
equip.damage.fillna('None', inplace=True)

In [15]:
dam=[]
damt=[]
for i in equip.damage:
    if len(i) ==4:
        dam.append('None')
        damt.append('None')
    else:
        i=json.loads(i.replace("'", '"'))
        dam.append(i['damage_dice'])
        damt.append(i['damage_type']['name'])

equip = equip.drop('damage', axis=1)
equip['damage']=dam
equip['damage_type']=damt

## Range

In [16]:
equip.range.fillna('None', inplace=True)

In [17]:
import ast
rang=[]

for i in equip.range:
    
    if i == 'None':
        rang.append('None')

    else:
        i=ast.literal_eval(i.replace("'", '"'))
        rang.append('Range: '+str(i['normal'])+';'+' Long: '+str(i['long']))
        
equip = equip.drop('range', axis=1)
equip['range']=rang

## Properties 

In [18]:
equip.properties[31]

"[{'name': 'Ammunition', 'url': '/api/weapon-properties/ammunition'}, {'name': 'Loading', 'url': '/api/weapon-properties/loading'}]"

In [19]:
equip.properties.fillna('None', inplace=True)

In [20]:
prop=[]
prop2=[]

for i in equip.properties:
   
    if (len(i)) == 4 or str(i) == "[]":
        prop2.append('None')

    else:
        i=json.loads(i.replace("'", '"'))
        for j in i:
            prop.append(j['name'])
            if len(prop)==len(i):
                prop2.append(prop)
                prop=[]        
        
prop3=[]

for i in prop2:
    i=str(i)
    i=i.replace("[","").replace("]","").replace("'","")
    prop3.append(i)

equip = equip.drop('properties', axis=1)
equip['properties']=prop3

## Armor class

In [21]:
equip.armor_class.value_counts()

{'base': 14, 'dex_bonus': True, 'max_bonus': 2}        2
{'base': 11, 'dex_bonus': True, 'max_bonus': None}     2
{'base': 16, 'dex_bonus': False, 'max_bonus': None}    1
{'base': 13, 'dex_bonus': True, 'max_bonus': 2}        1
{'base': 15, 'dex_bonus': True, 'max_bonus': 2}        1
{'base': 12, 'dex_bonus': True, 'max_bonus': 2}        1
{'base': 18, 'dex_bonus': False, 'max_bonus': None}    1
{'base': 14, 'dex_bonus': False, 'max_bonus': None}    1
{'base': 2, 'dex_bonus': False, 'max_bonus': None}     1
{'base': 17, 'dex_bonus': False, 'max_bonus': None}    1
{'base': 12, 'dex_bonus': True, 'max_bonus': None}     1
Name: armor_class, dtype: int64

In [22]:
equip.armor_class.fillna('None', inplace=True)

In [23]:
import ast
clas=[]

for i in equip.armor_class:
    
    if i == 'None':
        clas.append('None')

    else:
        i=ast.literal_eval(i.replace("'", '"'))
        clas.append('Base: '+str(i['base'])+';'+' Max bonus: '+str(i['max_bonus']))

equip = equip.drop('armor_class', axis=1)
equip['armor_class']=clas

## Speed 

In [24]:
equip.speed.value_counts()

{'quantity': 40, 'unit': 'ft/round'}    6
{'quantity': 60, 'unit': 'ft/round'}    2
{'quantity': 50, 'unit': 'ft/round'}    1
{'quantity': 4, 'unit': 'mph'}          1
{'quantity': 1, 'unit': 'mph'}          1
{'quantity': 3, 'unit': 'mph'}          1
{'quantity': 1.5, 'unit': 'mph'}        1
{'quantity': 2, 'unit': 'mph'}          1
{'quantity': 2.5, 'unit': 'mph'}        1
Name: speed, dtype: int64

In [25]:
equip.speed.fillna('None', inplace=True)

In [26]:
import ast
spe=[]

for i in equip.speed:
    
    if i == 'None':
        spe.append('None')

    else:
        i=ast.literal_eval(i.replace("'", '"'))
        spe.append(str(i['quantity'])+' '+(i['unit']))

equip = equip.drop('speed', axis=1)
equip['speed']=spe

## Throw range 

In [27]:
equip.throw_range.value_counts()

{'normal': 20, 'long': 60}     6
{'normal': 30, 'long': 120}    1
{'normal': 5, 'long': 15}      1
Name: throw_range, dtype: int64

In [28]:
equip.throw_range.fillna('None', inplace=True)

In [29]:
import ast
ran=[]

for i in equip.throw_range:
    
    if i == 'None':
        ran.append('None')

    else:
        i=ast.literal_eval(i.replace("'", '"'))
        ran.append('Range: '+str(i['normal'])+';'+' Long: '+str(i['long']))

equip = equip.drop('throw_range', axis=1)
equip['throw_range']=ran

# Nulos

In [30]:
equip.tool_category.fillna('None', inplace=True)
equip.vehicle_category.fillna('None', inplace=True)
equip.weapon_category.fillna('None', inplace=True)
equip.weapon_range.fillna('None', inplace=True)
equip.category_range.fillna('None', inplace=True)
equip.armor_category.fillna('None', inplace=True)
equip.capacity.fillna('None', inplace=True)

## Eliminación de columnas / creación columnas / cambio nombre

En base a la información aportada y que sea relevante, se eliminan/crean:

In [31]:
equip['id_equipment']=[i for i in range(1,len(equip)+1)]
equip = equip.drop(equip.columns[0], axis=1)
equip = equip.drop('special', axis=1)
equip = equip.drop('stealth_disadvantage', axis=1)
equip = equip.drop('str_minimum', axis=1)
equip = equip.drop('2h_damage', axis=1)
equip = equip.drop('quantity', axis=1)
equip = equip.drop('url', axis=1)
equip = equip.drop('index', axis=1)
equip = equip.drop('_id', axis=1)
equip = equip.drop('equipment_category', axis=1)
equip = equip.drop('contents', axis=1)

In [36]:
equip = equip[['id_equipment', 'name', 'desc', 'category', 'gear_category','cost', 'weight', 'speed', 'range', 'capacity', 'properties', 'damage', 'damage_type', 'category_range', 'throw_range', 'weapon_category', 'weapon_range', 'armor_category', 'armor_class' , 'tool_category', 'vehicle_category' ]]

In [37]:
equip.head()

,id_equipment,name,desc,category,gear_category,cost,weight,speed,range,capacity,properties,damage,damage_type,category_range,throw_range,weapon_category,weapon_range,armor_category,armor_class,tool_category,vehicle_category
0,1,Abacus,None,Adventuring Gear,Standard Gear,2 gp,2.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2,Acid (vial),"As an action, you can splash the contents of t...",Adventuring Gear,Standard Gear,25 gp,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,3,Alchemist's fire (flask),"This sticky, adhesive fluid ignites when expos...",Adventuring Gear,Standard Gear,50 gp,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,4,Alchemist's supplies,These special tools include the items needed t...,Tools,NaN,50 gp,8.0,None,None,None,None,None,None,None,None,None,None,None,None,Artisan's Tools,None
4,5,Amulet,A holy symbol is a representation of a god or ...,Adventuring Gear,Holy Symbol,5 gp,1.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Exportación 

In [38]:
equip.to_csv('../data/clean/cl_equip.csv', index=False)

Finalmente, esta tabla será informativa ya que no puede relacionarse directamente con el resto de tablas.